In [ ]:
import pandas as pd

from util import Pipeline

p = Pipeline('configs')

In [ ]:
store = pd.HDFStore('data/pipeline.h5', mode='r')
store.keys()

In [ ]:
store['snohomish_targets']

In [ ]:
store.close()